In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
unique_countries = [
    "Afghanistan",
    "Albania",
    "Argentina",
    "Australia",
    "Austria",
    "Bangladesh",
    "Belarus",
    "Belgium",
    "Bolivia",
    "Bosnia & Herzegovina",
    "Brazil",
    "Cameroon",
    "Canada",
    "Chile",
    "China",
    "Colombia",
    "Croatia",
    "Cuba",
    "Czechia",
    "Denmark",
    "Ecuador",
    "Egypt",
    "Estonia",
    "Finland",
    "France",
    "Gambia",
    "Georgia",
    "Germany",
    "Greece",
    "Hong Kong",
    "Hungary",
    "Iceland",
    "India",
    "Iran",
    "Iraq",
    "Ireland",
    "Israel",
    "Italy",
    "Japan",
    "Kenya",
    "Latvia",
    "Lebanon",
    "Libya",
    "Liechtenstein",
    "Lithuania",
    "Luxembourg",
    "Mexico",
    "Mongolia",
    "Montenegro",
    "Morocco",
    "Netherlands",
    "New Zealand",
    "Nigeria",
    "Norway",
    "Pakistan",
    "Paraguay",
    "Peru",
    "Poland",
    "Portugal",
    "Qatar",
    "Romania",
    "Russia",
    "Saudi Arabia",
    "Serbia",
    "Singapore",
    "Slovakia",
    "Slovenia",
    "South Africa",
    "South Korea",
    "Spain",
    "Sweden",
    "Switzerland",
    "Thailand",
    "Tunisia",
    "Turkey",
    "Ukraine",
    "United Kingdom",
    "United States",
    "Uruguay",
    "Venezuela",
]

In [7]:
df = pd.read_csv("trends/db_0.csv")
df["date"]= pd.to_datetime(df["date"])
df.dropna(inplace=True)

In [8]:
len(df)

292160

In [9]:
# Keep country and Facebook
df = df[["country", "Facebook"]]

In [10]:
# Get a list of countries
countries = df["country"].unique()

In [ ]:
# Create a new DataFrame 
new_df = pd.DataFrame()
for country in countries:
    tmp = df[df["country"] == country]#
    tmp = tmp.drop(columns=["country"])
    tmp = tmp.rename(columns={"Facebook": country})
    new_df = pd.concat([new_df.reset_index(drop=True), tmp.reset_index(drop=True)], axis=1)

In [ ]:
# Remove Global, and insert it at the front
new_df["Global"] = new_df.sum(axis=1) / new_df.shape[1]
GLOB = new_df.pop('Global')
new_df.insert(0, "Global", GLOB)

In [ ]:
VIZUALIZE = False

if VIZUALIZE:
    plt.figure(figsize=(60, 40), dpi=300)
    plt.title("Facebook")
    sns.heatmap(new_df.corr(), annot=True)

In [ ]:
CORRELATION_MATRIX = new_df.corr()

In [ ]:
# Use CORRELATION_MATRIX to whatever you want
print(CORRELATION_MATRIX.columns)
print(CORRELATION_MATRIX.values)


Index(['Global', 'Big Data', 'Artificial Intelligence', 'Augmented Reality',
       'Internet of Things', 'Cloud Computing', 'Big Data',
       'Artificial Intelligence', 'Augmented Reality', 'Internet of Things',
       ...
       'Big Data', 'Artificial Intelligence', 'Augmented Reality',
       'Internet of Things', 'Cloud Computing', 'Big Data',
       'Artificial Intelligence', 'Augmented Reality', 'Internet of Things',
       'Cloud Computing'],
      dtype='object', length=386)
[[ 1.          0.14125819  0.10457621 ...         nan         nan
   0.04567184]
 [ 0.14125819  1.          0.03008742 ...         nan         nan
  -0.01769648]
 [ 0.10457621  0.03008742  1.         ...         nan         nan
   0.02597278]
 ...
 [        nan         nan         nan ...         nan         nan
          nan]
 [        nan         nan         nan ...         nan         nan
          nan]
 [ 0.04567184 -0.01769648  0.02597278 ...         nan         nan
   1.        ]]


In [ ]:
def get_corr_matrix(filename,category,glob=False):
    '''
    filename: Trends/filename (remember .csv). ex: "db_0.csv"
    category: ex "Facebook", "YouTube"
    returns list of all countries involved (all names of columns/rows) and correlation matrix 
    '''
    df = pd.read_csv(f"Trends/{filename}")
    df["date"]= pd.to_datetime(df["date"])
    df.dropna(inplace=True)
    
    # Keep country and Facebook
    df = df[["country", category]]
    
    # Get a list of countries
    countries = df["country"].unique()
    
    # Create a new DataFrame 
    new_df = pd.DataFrame()
    for country in countries:
        tmp = df[df["country"] == country]#
        tmp = tmp.drop(columns=["country"])
        tmp = tmp.rename(columns={category: country})
        new_df = pd.concat([new_df.reset_index(drop=True), tmp.reset_index(drop=True)], axis=1)
    
    if glob:
        # Remove Global, and insert it at the front
        new_df["Global"] = new_df.sum(axis=1) / new_df.shape[1]
        GLOB = new_df.pop('Global')
        new_df.insert(0, "Global", GLOB)
    
    new_df = new_df.sort_index(axis=1)
        
    CORRELATION_MATRIX = new_df.corr()
    
    return CORRELATION_MATRIX.columns, CORRELATION_MATRIX
    
cnt,mtr = get_corr_matrix("db_0.csv","Facebook")

In [ ]:
# get categories for each file
def get_categories(filenames):
    '''
    filenames: list of filenames
    returns categories as a 2D-list where each
    inner list refers to a filename
    '''
    categories = []
    for filename in filenames:
        df = pd.read_csv(f"Trends/{filename}")
        df["date"]= pd.to_datetime(df["date"])
        df.dropna(inplace=True)
        categories.append(df.columns[2:])
    return categories

In [ ]:
# prepare to get corr matrices
filenames = []
for i in range(2):
    filenames += [f"db_{i}.csv"]

categories = get_categories(filenames)
print(filenames,categories)


['db_0.csv', 'db_1.csv'] [Index(['Facebook', 'YouTube', 'Instagram', 'Twitter', 'Snapchat'], dtype='object'), Index(['Elon Musk', 'Jeff Bezos', 'Bill Gates', 'Warren Buffett',
       'Mark Zuckerberg'],
      dtype='object')]


In [ ]:
# I'll store correlation matrices as 2D lists containing
# [category,countries,matrix]
# Takes less than 2 min to run
corr_matrices = []
for i in range(len(filenames)):
    for cat in categories[i]:
        countries, corr_matrix = get_corr_matrix(filenames[i],cat)
        corr_matrices.append([cat,countries,corr_matrix])

corr_matrices

[['Facebook',
  Index(['Afghanistan', 'Albania', 'Argentina', 'Australia', 'Austria',
         'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia & Herzegovina',
         'Brazil', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Croatia',
         'Cuba', 'Czechia', 'Denmark', 'Ecuador', 'Egypt', 'Estonia', 'Finland',
         'France', 'Gambia', 'Georgia', 'Germany', 'Greece', 'Hong Kong',
         'Hungary', 'Iceland', 'India', 'Iran', 'Iraq', 'Ireland', 'Israel',
         'Italy', 'Japan', 'Kenya', 'Latvia', 'Lebanon', 'Libya',
         'Liechtenstein', 'Lithuania', 'Luxembourg', 'Mexico', 'Mongolia',
         'Montenegro', 'Morocco', 'Netherlands', 'New Zealand', 'Nigeria',
         'Norway', 'Pakistan', 'Paraguay', 'Peru', 'Poland', 'Portugal', 'Qatar',
         'Romania', 'Russia', 'Saudi Arabia', 'Serbia', 'Singapore', 'Slovakia',
         'Slovenia', 'South Africa', 'South Korea', 'Spain', 'Sweden',
         'Switzerland', 'Thailand', 'Tunisia', 'Turkey', 'Ukraine',
  

In [ ]:
def convert_matrix(corr_matrix,treshold):
    '''
    corr_matrix: correlation matrix (dataframe)
    treshold: value between 0 and 1
    converts matrix to a matrix where all values > treshold gets replaced with 1 and the rest 0
    '''
    matrix = corr_matrix.copy()
    matrix[matrix>=treshold] = 1
    matrix[matrix<treshold] = 0
    
    return matrix


mat = convert_matrix(test_matrix,0.5)

In [ ]:
def make_groups_from_matrix(corr_matrix,treshold):
    '''
    corr_matrix: correlation matrix (dataframe)
    treshold: value between 0 and 1
    returns list of lists, where each list is a group
    '''
    matrix = convert_matrix(corr_matrix,treshold)
    
    groups = []
    used_countries = []
    countries = matrix.columns
    for c in countries:
        if c not in used_countries:
            # if country not visited before, make a new group and make sure all correlated countries are added
            group = [c]
            used_countries.append(c)             
            
            for ct in group:
                # make list of countries that ct correlates with
                # append all countries that has not been visited before to group and used_countries
                ct_corr = matrix[ct][matrix[ct]==1].index.values
                for ctr in ct_corr:
                    if ctr not in used_countries:
                        used_countries.append(ctr)
                        group.append(ctr)
            
            
            groups.append(group)
    return groups
            
    
test_matrix = corr_matrices[1][2]   
tull = make_groups_from_matrix(test_matrix,0.9)
tull

[['Afghanistan'],
 ['Albania',
  'Bosnia & Herzegovina',
  'Lebanon',
  'Turkey',
  'Serbia',
  'Bangladesh',
  'Israel'],
 ['Argentina'],
 ['Australia'],
 ['Austria',
  'France',
  'Switzerland',
  'Croatia',
  'Sweden',
  'Denmark',
  'Norway',
  'Czechia',
  'Tunisia'],
 ['Belarus'],
 ['Belgium', 'Italy', 'Chile', 'Germany'],
 ['Bolivia'],
 ['Brazil', 'Peru'],
 ['Cameroon'],
 ['Canada', 'Ireland', 'United Kingdom'],
 ['China'],
 ['Colombia'],
 ['Cuba'],
 ['Ecuador'],
 ['Egypt', 'India'],
 ['Estonia'],
 ['Finland'],
 ['Gambia'],
 ['Georgia'],
 ['Greece'],
 ['Hong Kong', 'Singapore'],
 ['Hungary'],
 ['Iceland'],
 ['Iran'],
 ['Iraq'],
 ['Japan'],
 ['Kenya'],
 ['Latvia'],
 ['Libya'],
 ['Liechtenstein'],
 ['Lithuania'],
 ['Luxembourg'],
 ['Mexico'],
 ['Mongolia'],
 ['Montenegro'],
 ['Morocco'],
 ['Netherlands'],
 ['New Zealand'],
 ['Nigeria'],
 ['Pakistan'],
 ['Paraguay'],
 ['Poland'],
 ['Portugal'],
 ['Qatar'],
 ['Romania'],
 ['Russia'],
 ['Saudi Arabia'],
 ['Slovakia'],
 ['Slovenia'],


In [ ]:
# make average correlation matrix

avg_corr_matrix = corr_matrices[0][2].copy()

for col in avg_corr_matrix.columns:
    avg_corr_matrix[col].values[:] = 0
print(avg_corr_matrix)
    
for i in range(50):
    i_matrix = corr_matrices[i][2].copy().fillna(1)
    avg_corr_matrix += i_matrix
print(avg_corr_matrix)

avg_corr_matrix /= 50
print(avg_corr_matrix)


                Afghanistan  Albania  Argentina  Australia  Austria  \
Afghanistan             0.0      0.0        0.0        0.0      0.0   
Albania                 0.0      0.0        0.0        0.0      0.0   
Argentina               0.0      0.0        0.0        0.0      0.0   
Australia               0.0      0.0        0.0        0.0      0.0   
Austria                 0.0      0.0        0.0        0.0      0.0   
...                     ...      ...        ...        ...      ...   
Ukraine                 0.0      0.0        0.0        0.0      0.0   
United Kingdom          0.0      0.0        0.0        0.0      0.0   
United States           0.0      0.0        0.0        0.0      0.0   
Uruguay                 0.0      0.0        0.0        0.0      0.0   
Venezuela               0.0      0.0        0.0        0.0      0.0   

                Bangladesh  Belarus  Belgium  Bolivia  Bosnia & Herzegovina  \
Afghanistan            0.0      0.0      0.0      0.0               

In [ ]:
overall_groups = make_groups_from_matrix(avg_corr_matrix,0.5)

for i in range(len(overall_groups)):
    df = pd.DataFrame(overall_groups[i])
    df.to_csv(f"groups/group_{i}.csv",index=False)
    
overall_groups    

[['Afghanistan'],
 ['Albania'],
 ['Argentina'],
 ['Australia'],
 ['Austria'],
 ['Bangladesh'],
 ['Belarus'],
 ['Belgium'],
 ['Bolivia'],
 ['Bosnia & Herzegovina'],
 ['Brazil'],
 ['Cameroon'],
 ['Canada', 'United Kingdom', 'United States'],
 ['Chile'],
 ['China'],
 ['Colombia'],
 ['Croatia'],
 ['Cuba'],
 ['Czechia'],
 ['Denmark'],
 ['Ecuador'],
 ['Egypt'],
 ['Estonia'],
 ['Finland'],
 ['France'],
 ['Gambia'],
 ['Georgia'],
 ['Germany'],
 ['Greece'],
 ['Hong Kong'],
 ['Hungary'],
 ['Iceland'],
 ['India'],
 ['Iran'],
 ['Iraq'],
 ['Ireland'],
 ['Israel'],
 ['Italy'],
 ['Japan'],
 ['Kenya'],
 ['Latvia'],
 ['Lebanon'],
 ['Libya'],
 ['Liechtenstein'],
 ['Lithuania'],
 ['Luxembourg'],
 ['Mexico'],
 ['Mongolia'],
 ['Montenegro'],
 ['Morocco'],
 ['Netherlands'],
 ['New Zealand'],
 ['Nigeria'],
 ['Norway'],
 ['Pakistan'],
 ['Paraguay'],
 ['Peru'],
 ['Poland'],
 ['Portugal'],
 ['Qatar'],
 ['Romania'],
 ['Russia'],
 ['Saudi Arabia'],
 ['Serbia'],
 ['Singapore'],
 ['Slovakia'],
 ['Slovenia'],
 ['Sou

In [ ]:
#3,6,7 (15-19,30-34,35-39)
corr_matrices[30][2].info()


<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, Afghanistan to nan
Data columns (total 80 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Afghanistan           79 non-null     float64
 1   Paraguay              79 non-null     float64
 2   Netherlands           79 non-null     float64
 3   Finland               79 non-null     float64
 4   Estonia               79 non-null     float64
 5   China                 79 non-null     float64
 6   Canada                79 non-null     float64
 7   United Kingdom        79 non-null     float64
 8   Belgium               79 non-null     float64
 9   Croatia               79 non-null     float64
 10  France                79 non-null     float64
 11  Cuba                  79 non-null     float64
 12  Ukraine               79 non-null     float64
 13  Denmark               79 non-null     float64
 14  Ecuador               79 non-null     float64
 15  Egypt              